In [ ]:
#External code comes essentially from my IBM ML course, Pytorch website 
#and various coding forums (including chatGPT) 

In [ ]:
# PyTorch Library
import torch 
# PyTorch Neural Network
import torch.nn as nn
# Allows us to transform data
import torchvision.transforms as transforms
# Allows us to get the digit dataset
import torchvision.datasets as dsets
# Creating graphs
import matplotlib.pylab as plt
# Allows us to use arrays to manipulate and store data
import numpy as np
# Memory monitoring
import psutil

import time
# Allows quantization from float32 to 16
import torch.quantization

In [ ]:
def show_data(data_sample):
    plt.imshow(data_sample[0].numpy().reshape(28, 28), cmap='gray')
    plt.title('y = ' + str(data_sample[1]))

### Download Train and Validation datasets

In [ ]:
train_dataset = dsets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
print("Print the training dataset:\n ", train_dataset)

In [ ]:
validation_dataset = dsets.MNIST(root='./data', download=True, train=False,transform=transforms.ToTensor())
print("Print the validation dataset:\n ", validation_dataset)

### Print first image in train and validation

In [ ]:
print("First Image and Label", show_data(train_dataset[0]))

In [ ]:
print("The label: ", train_dataset[0][1])

In [ ]:
print("First Image and Label", show_data(validation_dataset[0]))

## SoftMax Classifier Model

In [ ]:
# Inherits nn.Module which is the base class for all neural networks
class SoftMax(nn.Module):
    
    # Constructor
    def __init__(self, input_size, output_size):
        super(SoftMax, self).__init__()
        # Creates a layer of given input size and output size
        self.linear = nn.Linear(input_size, output_size)
        
    # Prediction
    def forward(self, x):
        # Runs the x value through the single layers defined above
        z = self.linear(x)
        return z

In [ ]:
train_dataset[0][0].shape

In [ ]:
#input is a 28 * 28 image, output is a probability array for each class
input_dim = 28 * 28
output_dim = 10

### Model Creation

In [ ]:
model = SoftMax(input_dim, output_dim)
print("The model:\n ", model)
print('W: ',list(model.parameters())[0].size())
print('b: ',list(model.parameters())[1].size())

### Try the model with random weights first

In [ ]:
# First we get the X value of the first image
X = train_dataset[0][0]
# We can see the shape is 1 by 28 by 28, we need it to be flattened to 1 by 28 * 28 (784)
print(X.shape)
X = X.view(-1, 28*28)
print(X.shape)
# Now we can make a prediction, each class has a value, and the higher it is the more confident the model is that it is that digit
model(X)

#### Define the learning rate, optimizer, criterion, and data loader

In [ ]:
learning_rate = 0.1
# The optimizer will updates the model parameters using the learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# The criterion will measure the loss between the prediction and actual label values
# This is where the SoftMax occurs, it is built into the Criterion Cross Entropy Loss
criterion = nn.CrossEntropyLoss()
# Created a training data loader so we can set the batch size
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100)
# Created a validation data loader so we can set the batch size
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=5000)

## Training

In [ ]:
n_epochs = 30
# Lists to keep track of loss and accuracy
loss_list = []
accuracy_list = []
speed_list=[]
memory_list=[]
size_list=[]
# Size of the validation data
N_test = len(validation_dataset)

#monitor memory usage
peak_gpu_memory_used =0
cpu_memory_used=0
# Function to train the model based on number of epochs
def train_model(n_epochs):
    # Loops n_epochs times
    for epoch in range(n_epochs):
        start_time = time.time()
        # For each batch in the train loader
        for x, y in train_loader:
            # Resets the calculated gradient value, this must be done each time as it accumulates if we do not reset
            optimizer.zero_grad()
            # Makes a prediction based on the image tensor
            z = model(x.view(-1, 28 * 28))
            # Calculates loss between the model output and actual class
            loss = criterion(z, y)
            # Calculates the gradient value with respect to each weight and bias
            loss.backward()
            # Updates the weight and bias according to calculated gradient value
            optimizer.step()
        
        # Each epoch we check how the model performs with data it has not seen which is the validation data, we are not training here
        correct = 0
        # For each batch in the validation loader
        for x_test, y_test in validation_loader:
            # Makes prediction based on image tensor
            z = model(x_test.view(-1, 28 * 28))
            # Finds the class with the higest output
            _, yhat = torch.max(z.data, 1)
            # Checks if the prediction matches the actual class and increments correct if it does
            correct += (yhat == y_test).sum().item()
        # Calculates the accuracy by dividing correct by size of validation dataset
        accuracy = correct / N_test
        # Keeps track loss
        loss_list.append(loss.data)
        # Keeps track of the accuracy
        accuracy_list.append(accuracy)
        peak_gpu_memory_used= torch.cuda.max_memory_allocated()/ (1024 ** 3)
        cpu_memory_used = psutil.virtual_memory().used / (1024 ** 3)
        end_time = time.time()
        epoch_time = end_time - start_time
        speed_list.append(epoch_time)
        memory_list.append(cpu_memory_used)

        print(f"Epoch {epoch+1}/{n_epochs} - Time: {epoch_time:.2f} seconds")
        print(f"Accuracy : {accuracy}")
        print()
        
#     print(f"Peak GPU Memory Used: {peak_gpu_memory_used:.2f} GB")
    print(f"CPU Memory Used: {cpu_memory_used:.2f} GB")

# Function call
train_model(n_epochs)

In [ ]:
def get_model_size(model):
    n_params=0
    params = np.sum(p.numel() for p in model.parameters())
    print("pparams:",params)

    size_bytes = params * 4  # Assuming float32 parameters (4 bytes per parameter)
    size_MB = size_bytes / (1024 ** 2)  # Convert to MB
    
                    
    
    return size_MB, n_params

model_size_1, params_number = get_model_size(model)
print(f"Model Size: {model_size_1:.2f} MB")


## Plot the loss and accuracy

In [ ]:
# Plot the loss and accuracy

fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.plot(loss_list,color=color)
ax1.set_xlabel('epoch',color=color)
ax1.set_ylabel('total loss',color=color)
ax1.tick_params(axis='y', color=color)
    
ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('accuracy', color=color)  
ax2.plot( accuracy_list, color=color)
ax2.tick_params(axis='y', color=color)
fig.tight_layout()

## Plot the speed and accuracy

In [ ]:
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.plot(speed_list,color=color)
ax1.set_xlabel('epoch',color=color)
ax1.set_ylabel('speed',color=color)
ax1.tick_params(axis='y', color=color)
    
ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('accuracy', color=color)  
ax2.plot( accuracy_list, color=color)
ax2.tick_params(axis='y', color=color)
fig.tight_layout()

## Plot memory and accuracy

In [ ]:
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.plot(memory_list,color=color)
ax1.set_xlabel('epoch',color=color)
ax1.set_ylabel('memory',color=color)
ax1.tick_params(axis='y', color=color)
    
ax2 = ax1.twinx()  
color = 'tab:blue'
ax2.set_ylabel('accuracy', color=color)  
ax2.plot( accuracy_list, color=color)
ax2.tick_params(axis='y', color=color)
fig.tight_layout()

## Test

### Before Quantization

In [ ]:
model.eval()

In [ ]:
x_test=validation_dataset[0][0]
x_test=x_test.view(-1, 28*28)

In [ ]:
model(x_test)

### Quantazition 

In [ ]:
quantized_model=torch.ao.quantization.quantize_dynamic(
    model,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.float16)  # the target dtype for quantized weights


In [ ]:
# After quantization is applied by going from float32 to float16, the model performance has not changed
quantized_model(x_test)

In [ ]:

#to fix: I was traying to get the size of the float16 model with this function
def get_model_size_f16(model):
    params = np.sum(p.numel() for p in model.parameters())
    print("pparams:",params)
    size_bytes = params * 2  # Assuming float16 parameters (2 bytes per parameter)
    print(f"size_bytes:{size_bytes}")
    size_MB = size_bytes / (1024 ** 2)  # Convert to MB
    
    return size_MB

In [ ]:
# Here too
def get_parameter_element_size(model):
    total_size_bytes = 0

    for param in model.parameters():
        element_size = param.element_size()
        total_size_bytes += element_size

    return total_size_bytes

In [ ]:
get_parameter_element_size(quantized_model)

In [ ]:
model_size_1 = get_model_size_f16(quantized_model)
print(f"Quantized Model Size: {model_size_1:.10f} MB")

## Trying to create a float16 version of the model from scratch and see if I can get the correct size (to be fixed)

In [ ]:
# create float16 model
class SoftMaxFloat16(nn.Module):
    # Constructor
    def __init__(self, input_dim, output_dim):
        super(SoftMaxFloat16, self).__init__()
        # Creates a layer of given input size and output size with float16 data type
        self.linear = nn.Linear(input_dim, output_dim).to(dtype=torch.float16)
        
    # Prediction
    def forward(self, x):
        # Runs the x value through the single layers defined above
        z = self.linear(x)
        return z

In [ ]:
model_float16 = SoftMaxFloat16(input_dim, output_dim)

In [ ]:
model_size_1 = get_model_size_f16(model_float16)
print(f"Quantized Model Size: {model_size_1:.10f} MB")

## I could get the size above(reduced by 2), but then the dtype on the training libs don't match...

In [ ]:
n_epochs = 10
# Lists to keep track of loss and accuracy
loss_list = []
accuracy_list = []
# Size of the validation data
N_test = len(validation_dataset)

#monitor memory usage
peak_gpu_memory_used =0
cpu_memory_used=0
# Function to train the model based on number of epochs
def train_model_16(n_epochs):
    # Loops n_epochs times
    for epoch in range(n_epochs):
        start_time = time.time()
        # For each batch in the train loader
        for x, y in train_loader:
            # Resets the calculated gradient value, this must be done each time as it accumulates if we do not reset
            optimizer.zero_grad()
            # Makes a prediction based on the image tensor
            z = model_float16(x.view(-1, 28 * 28))
            # Calculates loss between the model output and actual class
            loss = criterion(z, y)
            # Calculates the gradient value with respect to each weight and bias
            loss.backward()
            # Updates the weight and bias according to calculated gradient value
            optimizer.step()
        
        # Each epoch we check how the model performs with data it has not seen which is the validation data, we are not training here
        correct = 0
        # For each batch in the validation loader
        for x_test, y_test in validation_loader:
            # Makes prediction based on image tensor
            z = model_float16(x_test.view(-1, 28 * 28)) #MUST CHANGE DATA TYPE EVERYWHERE
            # Finds the class with the higest output
            _, yhat = torch.max(z.data, 1)
            # Checks if the prediction matches the actual class and increments correct if it does
            correct += (yhat == y_test).sum().item()
        # Calculates the accuracy by dividing correct by size of validation dataset
        accuracy = correct / N_test
        # Keeps track loss
        loss_list.append(loss.data)
        # Keeps track of the accuracy
        accuracy_list.append(accuracy)
        peak_gpu_memory_used= torch.cuda.max_memory_allocated()/ (1024 ** 3)
        cpu_memory_used = psutil.virtual_memory().used / (1024 ** 3)
        end_time = time.time()
        epoch_time = end_time - start_time

        print(f"Epoch {epoch+1}/{n_epochs} - Time: {epoch_time:.2f} seconds")
        print(f"Accuracy : {accuracy}")
        print()
        
#     print(f"Peak GPU Memory Used: {peak_gpu_memory_used:.2f} GB")
    print(f"CPU Memory Used: {cpu_memory_used:.2f} GB")

# Function call
train_model_16(n_epochs)